# load data

In [3]:
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse

In [26]:
from datetime import date
from datetime import timedelta

In [33]:
two_weeks_old = date.today() - timedelta(days=14)

In [35]:
str(two_weeks_old)

'2021-01-04'

In [43]:
cnx = sqlite3.connect("../home.db")
query = f"SELECT id, full_text, created_at FROM tweets WHERE created_at < '{str(two_weeks_old)}'"
print(query)
df_tweets = pd.read_sql_query(
    query,
    cnx,
)

SELECT id, full_text, created_at FROM tweets WHERE created_at < '2021-01-04'


In [44]:
df_tweets.head()

,id,full_text,created_at
0,2627602600,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00
1,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead...",2011-05-16T22:57:37+00:00
2,303962112307363840,If corporate America built its own low orbit i...,2013-02-19T20:19:49+00:00
3,320150923336892416,These two books contain the sum total of all h...,2013-04-05T12:28:23+00:00
4,334391242395906048,I really do enjoy following @Aelkus.,2013-05-14T19:34:19+00:00


In [45]:
df_tweets.tail()

,id,full_text,created_at
36953,1345881625062727680,I'm thinking about people who can afford to be...,2021-01-03T23:55:59+00:00
36954,1345881793442922497,and since they're somewhere between 26 and 34 ...,2021-01-03T23:56:39+00:00
36955,1345882147844853760,"So... I found this, on the IO box.\nA customer...",2021-01-03T23:58:03+00:00
36956,1345882354854727681,"R might be ""registers"". Given the size of the ...",2021-01-03T23:58:53+00:00
36957,1345882580663603201,*The end of the Republican Party would be the ...,2021-01-03T23:59:47+00:00


In [46]:
with open('news_domains.txt', 'r') as f:
    news_domains = json.loads(f.read())

# utils

In [48]:
def find_url(tweet):
    return re.findall("http\S+", tweet)

def clean_links(tweet):
    tweet = re.sub(r'bit.ly/\S+', '', tweet)
    tweet = re.sub(r't.co/\S+', '', tweet)
    tweet = re.sub(r'buff.ly/\S+', '', tweet)
    tweet = re.sub(r'twitter.com/\S+', '', tweet)
    return tweet

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain
    
def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return(l)

# function for df

In [49]:
def find_news(df):

    df["urls"] = df["full_text"].apply(find_url)
    df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)


    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())


    for col in new_columns_list:
        df[col] = df[col].isin(news_domains)

    df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)
    
    return df


In [50]:
%%time
df_tweets = find_news(df_tweets)

CPU times: user 446 ms, sys: 7.9 ms, total: 454 ms
Wall time: 457 ms


In [ ]:
# suffle them!

In [59]:
to_custom_news_feed = df_tweets[df_tweets["contains_news"] == 0]["id"].sample(frac=1).reset_index(drop=True)[:1000]

In [60]:
to_custom_news_feed

0      1337875267096875015
1      1335698027537969155
2      1345699105385959435
3      1336874318882467842
4      1337717825629253633
              ...         
995    1341640996686278662
996    1340790897630437376
997    1341883276487700481
998    1336753706012987399
999    1342002012204851201
Name: id, Length: 1000, dtype: int64

In [61]:
type(to_custom_news_feed)

pandas.core.series.Series

In [64]:
to_custom_news_feed.to_csv("first_batch.csv")